In [ ]:
# produce the list of the individual shapefiles for each of the regions along the project area 

In [1]:
%run C:\Users\rmcgr\Rangeland_monitoring\veg_classification\data_processing\list_of_files_in_dir.py --direc C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb \
    --exten .shp --txtfile C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\shp_list.csv

In [2]:
# the next block of code produces the list needed to input into the multi clip script which produces the image chips 
# from the aerial photogrpahy. 

In [3]:
import pandas as pd

"""
read in the list of shapefiles and clean up the list to input 
into the multi clip script
"""

# provide the path to the folder with the list of shapfiles
path = r"C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/"

# provide the path and name of the raster imagery being clipped.
img = r"C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/jenkins13_rtk_hover_80_no_gcp_mosaic_group1.tif"

data = os.path.join(path, "shp_list.csv")
print(data)
clip_list = pd.read_csv(data, header=None)
print(clip_list)
clip_list['img'] = img
clip_list.columns=['shp','img']
clip_list.head(3)
clip_list_final = clip_list[['img','shp']]
path = data[:-12]

print (path + 'clip_list.csv')
clip_list_final.to_csv(path + 'clip_list.csv')

C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/shp_list.csv
            0
0  j13_01.shp
1  j13_02.shp
C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/clip_list.csv


In [4]:
# cd to the directory containing all of the individual shapefiles

In [5]:
cd C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb

C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb


In [6]:
# run the script which clips out all of the image chips 
# this script renames each individual image chip using the unique id in the shapefile name

In [7]:
%run C:/Users/rmcgr/Rangeland_monitoring/veg_classification/data_processing/multi_clip_tool_rasterio.py -i C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/clip_list.csv

32752
32752


In [8]:
# create a list of image chips to be classifed 

In [9]:
%run C:\Users\rmcgr\Rangeland_monitoring\veg_classification\data_processing\list_of_files_in_dir.py --direc C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb \
    --exten .tif --txtfile C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\img_list.csv

In [10]:
# cd to the directory containing the classifier 

In [11]:
cd C:\Users\rmcgr\Rangeland_monitoring\veg_classification\aerial_photo_class

C:\Users\rmcgr\Rangeland_monitoring\veg_classification\aerial_photo_class


In [12]:
# run the help function on the script which applies the rgb aerial photograph classifier to each of the image chips
# this script calls a script which applies the classifer to individual imagery. 

In [13]:
%run apply_rgb_comb_classifier_multi.py -h

usage: apply_rgb_comb_classifier_multi.py [-h] [-s IMGLIST] [-d DIREC]
                                          [-c CSV]

optional arguments:
  -h, --help            show this help message and exit
  -s IMGLIST, --imglist IMGLIST
                        provide the path and name of the csv file with the
                        list of imagery to process
  -d DIREC, --direc DIREC
                        path to the directory containing the imagery
  -c CSV, --csv CSV     provide the path to the directory and name of the csv
                        file containing the results


In [14]:
# input the command line arguments requried to run the script
# this script outputs the classified image chip and also extracts out the FPC value 
# the FPC values for each image chip are saved out to a csv file 

In [39]:
from __future__ import print_function, division

# import the requried modules
import sys
import os
import argparse
import pdb
import pandas as pd
import csv
import subprocess
import rasterio
import numpy as np


# command arguments 
def getCmdargs():

    p = argparse.ArgumentParser()

    p.add_argument("-s","--imglist", help="provide the path and name of the csv file with the list of imagery to process")
    
    p.add_argument("-d","--direc", help="path to the directory containing the imagery")
    
    p.add_argument("-c","--csv", help="provide the path to the directory and name of the csv file containing the results")
    
    cmdargs = p.parse_args()
    
    # if there is no image list the script will terminate
    if cmdargs.imglist is None:

        p.print_help()

        sys.exit()

    return cmdargs


def applyModel(imglist,directory):
    
    """
    produce the classified image from a image chip representing a site 
    and extract out the total fpc value for the plot. 
    Save the image chip and save out the fpc results as a csv file
    """
    
    site_list = []
    fpc_value_list = []
    
    
    # open the list of imagery and read it into memory
    df = pd.read_csv(imglist,header=None)
    print(df)
    for index, row in df.iterrows():
        rgb_image = os.path.join(directory + (str(row[0])))
        print(rgb_image)
        
        fileN = (str(row[0]))
        
        outfile = os.path.join(directory, "{0}_{1}".format(fileN[:-4], 'rgb_comb_class2.tif'))
        
        #print (fileN)
        #print (outfile)         
 
        # call and run the vegetation index scripts
        # to get this to work on windows the code being called by os.system needs to be stored 
        # and run from the same directory. 
    
#         import rgb_digital_aerial_photo_classifier
#         rgb_digital_aerial_photo_classifier.maun_routine(rgb_image,outfile)
        
        print('up to here')
        #import rgb_digital_aerial_photo_classifier2
        #rgb_digital_aerial_photo_classifier2.main_routine(rgb_image,outfile)
        cmd = "python rgb_digital_aerial_photo_classifier.py --reffile %s --outfile %s"% (rgb_image,outfile) 
        subprocess.call(cmd, shell=True)    
        os.system(cmd)
        print (outfile + ' complete')
        
        # read in the classified image and calculate the estimated fpc
        # read in with rasterio
        dataset = rasterio.open(outfile)
        # read in the first band as numpy array
        band1 = dataset.read(1)
        
        # get the shape of the image chip which to calculate the percentage fpc
        numPixels = (band1.shape[0] * band1.shape[1])
        
        # get the number of pixels classifed as green mangroves and calculate the % FPC for the site 
        greenM = band1[band1 == 1]
        greenMc = (greenM.shape[0]) 
        fpc = (greenMc / numPixels)*100
        
        site_list.append(fileN)
        
        fpc_value_list.append(fpc)
        
    return (site_list, fpc_value_list)   
           
# calls in the command arguments and applyModel function.        

    
#cmdargs = getCmdargs()
directory = r"C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb"
csvfile = r"C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\classify_J13_80_no_gcp_rgb_results.csv"
imglist = r"C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\img_list.csv"

site_list, fpc_value_list = applyModel(imglist,directory)

# save out the results to a csv file
data = list(zip(site_list, fpc_value_list))

results = pd.DataFrame(data,columns=['site','fpc'])

results.to_csv(csvfile)

                                                 0
0                                       j13_01.tif
1                                       j13_02.tif
2  jenkins13_rtk_hover_80_no_gcp_mosaic_group1.tif
C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgbj13_01.tif
up to here
C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\j13_01_rgb_comb_class2.tif complete


RasterioIOError: C:\Users\rmcgr\Desktop\drone_class_copy\classify_J13_80_no_gcp_rgb\j13_01_rgb_comb_class2.tif: No such file or directory

In [15]:
%run apply_rgb_comb_classifier_multi.py -s C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/img_list.csv   \
    -d C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/ \
        -c C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/classify_J13_80_no_gcp_rgb_results_test.csv

C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/j13_01_rgb_comb_class2.tif complete


RasterioIOError: C:/Users/rmcgr/Desktop/drone_class_copy/classify_J13_80_no_gcp_rgb/j13_01_rgb_comb_class2.tif: No such file or directory

In [ ]:
#  

In [ ]:
%run rgb_digital_aerial_photo_classifier.py -h

In [ ]:
%run rgb_digital_aerial_photo_classifier.py --reffile C:/DATA/todd/example/shp/grd_50m4_ID__100.tif --outfile C:/DATA/todd/example/shp/grd_50m4_ID__100_class.tif